In [1]:
from gurobipy import Model, GRB, tuplelist, quicksum
from gurobipy import read as gurobi_read_model
from multi_run_scheduler import *
import json
import random
import math
import os
import copy
import time

In [2]:
t1 = time.time()
s = SchedulerSimulation("sample_input/sample_input_v2_0.json")
s.run_simulation()
t2 = time.time()
print(f"{round(t2 - t1, 3)} seconds")

START - CURRENT RESOURCES {'t1'}
7 Events loaded.
Running after 1 events...
Scheduling Run Information:
Now: 0
Current Requests: ['0', '1', '2', '3', '4']
Completed Requests: set()
Schedulable Requests: dict_keys(['0', '1', '2', '3', '4'])
Current Resources: {'t1'}
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-20
Set parameter MIPGap to value 0.01
Set parameter Method to value 3
Model constructed
Model optimized
---
Total Priority: 7457.614210388017
Scheduled Observations: 5
RequestID: 2, Resource: t1, S/E(D): 4970/6079(1109), Priority: 241.3791076829712
RequestID: 1, Resource: t1, S/E(D): 51264/61794(10530), Priority: 2273.1580202294244
RequestID: 3, Resource: t1, S/E(D): 79975/80543(568), Priority: 264.37209823821155
RequestID: 4, Resource: t1, S/E(D): 112918/115175(2257), Priority: 1050.4865491798412
RequestID: 0, Resource: t1, S/E(D): 220537/228284(7747), Priority: 3627.7184350575685
---

Running after 2 events...
Scheduling Run Information